In [ ]:
import os
import sys
import csv
import nltk
import re
import tensorflow as tf
import count_data as cd

from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from gensim import corpora
from nltk import pos_tag
from nltk.corpus import wordnet
from collections import defaultdict
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
def load_data(url=r'C:\python_project\ML\clean_data\clean_data_0.5_no_repeat.csv', split_num=0.8):
#  加载数据
    dataset = []
    with open(url, 'r', encoding='gbk') as rf:
        reader = csv.reader(rf, dialect=csv.excel)
        for row in reader:
            dataset.append({'id': row[1], 'text': row[2], 'type': row[4]})
#  词性标注方法    
    def get_wordnet_pos(tag):
        if tag.startswith('J'):
            return wordnet.ADJ
        elif tag.startswith('V'):
            return wordnet.VERB
        elif tag.startswith('N'):
            return wordnet.NOUN
        elif tag.startswith('R'):
            return wordnet.ADV
        else:
            return None
    
    wnl = WordNetLemmatizer()
#  制作数据集            
    sample = []
    label = []
    y = []
    for data in dataset:
        sample.append(data['text'])
        y.append(data['type'])
    for all_label in y:
        all_label = eval(all_label)
        label.append(all_label)
    label = tf.constant(label)
    label = tf.cast(label, dtype=tf.float32)
#  替换文本中的地址、邮件、电话号码以及数字    
    url = r'((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))'
    mail = r'([\w\d_\.\-]+)@([\w\d\-]+)(\.[\w\d\-]+)*(\.[\w\d]{2,6})'
    phone = re.compile(r'''( (\d{3}|\(\d{3}\))?(\s|-|\.)? \d{3}(\s|-|\.)\d{4}(\s*(ext|x|ext.)\s*\d{2,5})?)''',re.VERBOSE)
    num = r'-?[1-9]\d*|-?([1-9]\d*\.\d*|0\.\d*[1-9]\d*|0?\.0+|0)$'

    sample = [re.sub(url,'TAGURL',text) for text in sample]
    sample = [re.sub(mail,'TAGMAIL',text) for text in sample]
    sample = [re.sub(phone,'TAGPHONE',text) for text in sample]
    sample = [re.sub(num,'TAGNUM',text) for text in sample]
#  分词
    tokenizer = Tokenizer(oov_token='<OOV>', lower=True)
    tokenizer.fit_on_texts(sample)
    word_index = tokenizer.word_index
    sample = tokenizer.texts_to_sequences(sample)
    sample = tokenizer.sequences_to_texts(sample)
    
    sample = [segment.split() for segment in sample]
#  去除停用词    
    stoplist = set('for a of the and to'.split()) 
    sample = [[word for word in segment if word not in stoplist]
             for segment in sample]
    
    sample = [pos_tag(segment) for segment in sample]
#  用lemmatize将每个词进行词形还原    
    new_datasets = []
    for segment in sample:
        save = []
        for word in segment:
            save.append(wnl.lemmatize(word[0], get_wordnet_pos(word[1]) or wordnet.NOUN))
        new_datasets.append(' '.join(save))
    
#  划分训练集与测试集并返回    
    x_train = new_datasets[:int(len(new_datasets)*split_num)]
    x_test = new_datasets[int(len(new_datasets)*split_num):]
    y_train = label[:int(len(label)*split_num)]
    y_test = label[int(len(label)*split_num):]
    
    return (x_train, y_train), (x_test, y_test)

In [ ]:
def transform_to_multi_class(label, class_number=12):
    new_label = list(map(lambda a:[], range(class_number)))
    for all_label in label:
        for i in range(len(all_label)):
            new_label[i].append(all_label[i].numpy())
    new_label = tf.constant(new_label)
    return new_label

原始的文本对应的标签为\[0 1 1 0... ...0 1 0\]，用于多标签分类

将该标签进行转换依次提取每个位置的数字作为每个二分类器的标签

In [ ]:
def split_word(text):
    #  预处理分词
    punctuation = r'!\"#$%&()*+,.\-/:;<=>?@[\]^_`{|}~'
    stoplist = set('for a of the and to'.split())   
    wnl = WordNetLemmatizer()
    text = [re.sub(r'[{}]+'.format(punctuation),' ',segment) for segment in text]    
    texts = [[word for word in segment.lower().split() if word not in stoplist]
             for segment in text]
    frequency = defaultdict(int)
    for word in texts:
        for token in word:
            frequency[token] += 1
    texts = [[token for token in word if frequency[token] > 0]
             for word in texts]
#    texts = [nltk.pos_tag(text) for text in texts]
#    texts = [[wnl.lemmatize(word[0], word[1]) for word in text] for text in texts]
    return texts

分词操作方法，在实际使用中没有用上